# Thoughts on Pytest

## Your experience
* Have you used unittest?
* Have you used pytest?
* What other testing frameworks have you used, not just in Python?

## My experience
* I have used unittest a lot on StaGE & Improver
* I love doctest
* I have only come across pytest in the last week
* I have a PR which if accepted will add pytests to Rose.

<aside class="warning"><font color="red"><h3>I'm not an expert, but a fellow traveller sharing discoveries - I could be plain wrong, or raise points people disagree with!
    </font></aside></h3>

## What I found on the Web:
I'm summarizing, doesn't mean arguments necessarily convince me...
### The web says (In Favour):
* Produces tidier code than unittest - no requirement to subclass unittest.TestCase [1] [2] [5]

* Simpler to use because it uses `assert` rather than importing asserting functions.

* Failure info is often easier to read [1]

* Debugging is quite nice - it's possible to drop into debug mode if a test fails. [1] [5]

* Still Runs Unittests and Doctests tests anyway.

* Extensions/Plugins include support for Parallelization - really important on big projects! [4]

### ... and against
* Not making tests methods of a test class make it harder for test framework to decide what to run [3]

* unittest derives from xUnit and so has cross language familiarity? [1]

* Not available on RHEL6

### Sources
1. https://www.slant.co/versus/9148/9149/~unittest_vs_pytest
2. https://docs.python-guide.org/writing/tests/
3. https://cournape.github.io/why-i-am-not-a-fan-of-pytest.html - bit ranty?
4. https://docs.python-guide.org/writing/tests/
5. https://docs.pytest.org/en/latest/index.html

# Pytest - An example
For the sake of illustrating the differences to myself I wrote some very simple code:

In [2]:
# %load function.py
# A simple mathematical function on which to try different testing frameworks

def square_plus_10(x):
    """
    Doctest Example (Including one designed to fail)
    >>> square_plus_10(2)
    14
    >>> square_plus_10(-2)
    14
    >>> square_plus_10(7)
    42
    """
    return (x * x) + 10


def main():
    print("This main function doesn\'t do anything")


if __name__ == "__main__":
    main()
    

This main function doesn't do anything


I also wrote some unittests:


In [41]:
import unittest

from function import square_plus_10

TEST_CASES = ([ 2, 14],
              [-2, 14],
              [ 7, "Bowl of petunias"])


# Very simple unittest
class TestSquarePlus10(unittest.TestCase):
    def test_basic(self):
        self.assertEqual(square_plus_10(2), 14)

    def test_basic(self):
        self.assertEqual(square_plus_10(-2), 14)

    def test_basic(self):
        self.assertEqual(square_plus_10(7), "Bowl of petunias")

which I can run thus:

In [37]:
!python3 -m unittest tests_unittest/test_function1.py

F
FAIL: test_basic (tests_unittest.test_function1.TestSquarePlus10)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/net/my/home/dir/tutself_pytest/tests_unittest/test_function1.py", line 21, in test_basic
    self.assertEqual(square_plus_10(7), "Bowl of petunias")
AssertionError: 59 != 'Bowl of petunias'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (failures=1)


In [42]:
import unittest

from function import square_plus_10

TEST_CASES = ([ 2, 14],
              [-2, 14],
              [ 7, "Bowl of petunias"])

# Slightly More Convient
class TestSquarePlus10_iterative_but_unhelpful(unittest.TestCase):
    def test_basic(self):
        for case in TEST_CASES:
            # create a useful error message
            msg = f"square_plus_10({case[0]}) != {case[1]}"
            self.assertEqual(square_plus_10(case[0]), case[1], msg)

In [44]:
!python3 -m unittest tests_unittest/test_function2~.py

F
FAIL: test_basic (tests_unittest.test_function2.TestSquarePlus10_iterative_but_unhelpful)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/net/my/home/dir/tutself_pytest/tests_unittest/test_function2.py", line 17, in test_basic
    self.assertEqual(square_plus_10(case[0]), case[1], msg)
AssertionError: 59 != 'Bowl of petunias' : square_plus_10(7) != Bowl of petunias

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (failures=1)


In [46]:
import unittest

from function import square_plus_10

TEST_CASES = ([ 2, 14],
              [-2, 14],
              [ 7, "Bowl of petunias"])


# Much nicer (from 3.5 onwards)
class TestSquarePlus10_iterative_nicer(unittest.TestCase):
    def test_basic(self):
        for case in TEST_CASES:
            with self.subTest(case):
                self.assertEqual(square_plus_10(case[0]), case[1])

In [47]:
!python3 -m unittest tests_unittest/test_function3.py


FAIL: test_basic (tests_unittest.test_function3.TestSquarePlus10_iterative_nicer) [[7, 'Bowl of petunias']]
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/net/my/home/dir/tutself_pytest/tests_unittest/test_function3.py", line 17, in test_basic
    self.assertEqual(square_plus_10(case[0]), case[1])
AssertionError: 59 != 'Bowl of petunias'

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (failures=1)


we can also run these tests using pytest, which, quelle surprise, produces the same results...

In [24]:
! pytest tests_unittest/

=============================================================================================== test session starts ===============================================================================================
platform linux -- Python 3.6.6, pytest-4.2.1, py-1.7.0, pluggy-0.8.1
rootdir: /net/my/home/dir/tutself_pytest, inifile:
plugins: xdist-1.26.1, remotedata-0.3.1, openfiles-0.3.2, forked-1.0.2, doctestplus-0.2.0, arraydiff-0.3
collected 3 items                                                                                                                                                                                                 

tests_unittest/test_function.py FFF                                                                                                                                                                         [100%]

==================================================================================================== FAILURES =============================

We can also write a similar test in pure pytest format (which means that it won't work in unittest)

In [48]:
import unittest
import pytest

from function import square_plus_10

TEST_CASES = ([ 2, 14],
              [-2, 14],
              [ 7, "Bowl of petunias"])

# Much nicer
# No class required, no self required, no subTest required
# class TestSquarePlus10_iterative_nicer(unittest.TestCase):
def test_pure_pytest_example():
    for case in TEST_CASES:
        assert(square_plus_10(case[0]) == case[1])


# Using Pytest parameterize
@pytest.mark.parametrize("test_input, expected", TEST_CASES)
def test_pytest_with_parameterization(test_input, expected):
    assert square_plus_10(test_input) == expected


# Show the working of pytest.approx lest anyone ask me about
# assertAlmostEqual
def test_pytest_almost_equal():
    pytest.approx(10, abs=0.1) == 9.9999
    pytest.approx(1, rel=0.1) == 1.01

In [33]:
!pytest tests_pytest/test_pytest_example.py

=============================================================================================== test session starts ===============================================================================================
platform linux -- Python 3.6.6, pytest-4.2.1, py-1.7.0, pluggy-0.8.1
rootdir: /net/my/home/dir/tutself_pytest, inifile:
plugins: xdist-1.26.1, remotedata-0.3.1, openfiles-0.3.2, forked-1.0.2, doctestplus-0.2.0, arraydiff-0.3
collected 5 items                                                                                                                                                                                                 

tests_pytest/test_pytest_example.py F..F.                                                                                                                                                                   [100%]

==================================================================================================== FAILURES =============================

* We didn't have to do any of the self.xxx stuff
* We didn't have to have classes (although we can if it helps us organise stuff)
* We didn't have to do much to get a useful error message.

## Doctests
cooler still you can run doctests:

In [50]:
!pytest --doctest-modules function.py

=============================================================================================== test session starts ===============================================================================================
platform linux -- Python 3.6.6, pytest-4.2.1, py-1.7.0, pluggy-0.8.1
rootdir: /net/my/home/dir/tutself_pytest, inifile:
plugins: xdist-1.26.1, remotedata-0.3.1, openfiles-0.3.2, forked-1.0.2, doctestplus-0.2.0, arraydiff-0.3
collected 1 item                                                                                                                                                                                                  

function.py F                                                                                                                                                                                               [100%]

==================================================================================================== FAILURES =============================

# Python DeBug Compatibility
You can even drop into the doctest in pdb mode!

In [25]:
!pytest --doctest-modules function.py --pdb

=============================================================================================== test session starts ===============================================================================================
platform linux -- Python 3.6.6, pytest-4.2.1, py-1.7.0, pluggy-0.8.1
rootdir: /net/my/home/dir/tutself_pytest, inifile:
plugins: xdist-1.26.1, remotedata-0.3.1, openfiles-0.3.2, forked-1.0.2, doctestplus-0.2.0, arraydiff-0.3
collected 1 item                                                                                                                                                                                                  

function.py F
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> traceback >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
004 
005     Doctest Example (Including one designed to fail)
006     >>> square_plus_10(2)
007     14
008     >>> square_plu

## Markers

A small change to the pytests file allows us to tag tests
```python
@pytest.mark.pointless_tests
def test_pytest_almost_equal():
```

In [53]:
!pytest tests_pytest/test_pytest_example_allgood.py -m "pointless_tests"

=============================================================================================== test session starts ===============================================================================================
platform linux -- Python 3.6.6, pytest-4.2.1, py-1.7.0, pluggy-0.8.1
rootdir: /net/my/home/dir/tutself_pytest, inifile:
plugins: xdist-1.26.1, remotedata-0.3.1, openfiles-0.3.2, forked-1.0.2, doctestplus-0.2.0, arraydiff-0.3
collected 2 items / 1 deselected / 1 selected                                                                                                                                                                     

tests_pytest/test_pytest_example_allgood.py .                                                                                                                                                               [100%]

===================================================================================== 1 passed, 1 deselected in 0.01 seconds ==============

In [54]:
!pytest tests_pytest/test_pytest_example_allgood.py -m "not pointless_tests"

=============================================================================================== test session starts ===============================================================================================
platform linux -- Python 3.6.6, pytest-4.2.1, py-1.7.0, pluggy-0.8.1
rootdir: /net/my/home/dir/tutself_pytest, inifile:
plugins: xdist-1.26.1, remotedata-0.3.1, openfiles-0.3.2, forked-1.0.2, doctestplus-0.2.0, arraydiff-0.3
collected 2 items / 1 deselected / 1 selected                                                                                                                                                                     

tests_pytest/test_pytest_example_allgood.py .                                                                                                                                                               [100%]

===================================================================================== 1 passed, 1 deselected in 0.02 seconds ==============

or use k to get specific expression matching.

In [59]:
!pytest tests_pytest/test_pytest_example_allgood.py -k "almost_equal"

=============================================================================================== test session starts ===============================================================================================
platform linux -- Python 3.6.6, pytest-4.2.1, py-1.7.0, pluggy-0.8.1
rootdir: /net/my/home/dir/tutself_pytest, inifile:
plugins: xdist-1.26.1, remotedata-0.3.1, openfiles-0.3.2, forked-1.0.2, doctestplus-0.2.0, arraydiff-0.3
collected 2 items / 1 deselected / 1 selected                                                                                                                                                                     

tests_pytest/test_pytest_example_allgood.py .                                                                                                                                                               [100%]

===================================================================================== 1 passed, 1 deselected in 0.01 seconds ==============

...or select test classes or class.methods()...

In [64]:
!pytest tests_unittest/test_function3.py::TestSquarePlus10_iterative_nicer::test_basic

=============================================================================================== test session starts ===============================================================================================
platform linux -- Python 3.6.6, pytest-4.2.1, py-1.7.0, pluggy-0.8.1
rootdir: /net/my/home/dir/tutself_pytest, inifile:
plugins: xdist-1.26.1, remotedata-0.3.1, openfiles-0.3.2, forked-1.0.2, doctestplus-0.2.0, arraydiff-0.3
collected 1 item                                                                                                                                                                                                  

tests_unittest/test_function3.py F                                                                                                                                                                          [100%]

==================================================================================================== FAILURES =============================

And, rather wonderfully:
```python
@pytest.mark.skip("Some Excuse")
...
@pytest.mark.skipif(some>condition, "Another Excuse") # Default example - test Python version
```

## Print
`pytest --capture=no` will cause all print() to go to sdout.

## Pydest Xdist -
A plugin to allow you to use multiprocessor testing:

In [30]:
!pytest tests_pytest/test_parallel.py -n 1

=============================================================================================== test session starts ===============================================================================================
platform linux -- Python 3.6.6, pytest-4.2.1, py-1.7.0, pluggy-0.8.1
rootdir: /net/my/home/dir/tutself_pytest, inifile:
plugins: xdist-1.26.1, remotedata-0.3.1, openfiles-0.3.2, forked-1.0.2, doctestplus-0.2.0, arraydiff-0.3
gw0 [4]
....                                                                                                                                                                                                        [100%]
============================================================================================ 4 passed in 18.93 seconds ============================================================================================


In [31]:
!pytest tests_pytest/test_parallel.py -n 4

=============================================================================================== test session starts ===============================================================================================
platform linux -- Python 3.6.6, pytest-4.2.1, py-1.7.0, pluggy-0.8.1
rootdir: /net/my/home/dir/tutself_pytest, inifile:
plugins: xdist-1.26.1, remotedata-0.3.1, openfiles-0.3.2, forked-1.0.2, doctestplus-0.2.0, arraydiff-0.3
gw0 [4] / gw1 [4] / gw2 [4] / gw3 [4]
....                                                                                                                                                                                                        [100%]
============================================================================================ 4 passed in 6.66 seconds =============================================================================================


## Conclusions
* There a lots of good reasons to use Pytest. 
* I've seen some doubts on the internet
* But not many
* I haven't touched on Pytest fixtures I propose adding them to the Python Guild Todo List!

## Discussion Point
1. What do people think of the web arguements?

2. Should we aim to make our pytests unittest compatible, or not worry?